# Sequential SAR Change Detection

In [6]:
from auxil.eeSar_seq import run
run()

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


VkJveChjaGlsZHJlbj0oVGV4dGFyZWEodmFsdWU9dSdJbWFnZXMgZm91bmQ6IDMwIFxuQWNxdWlzaXRpb24gZGF0ZXM6IFQyMDE4MDQwMy4uLlQyMDE4MDkzMFxuUmVsYXRpdmUgb3JiaXQgbnXigKY=
